## INICIALIZO EL CLUSTER PARA EJECUTAR SPARK

In [2]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AME Started!
Welcome to AME Version 3.3.9 (January 2022)

SparkSession available as spark and AME available as api

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## IMPORT LAS LIBRERIAS

In [3]:
from pyspark.sql.functions import count , col , lit, desc, when, exp , greatest , least , max , min , round

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## FUNCION GENERICA

In [4]:
def archive_date(table, date, partition_name):
    archive = spark.read.format('bigquery').option('table', table).load().filter("ARCHIVE <= '{fecha} 00:00:00'".format(fecha = date)).select(partition_name).distinct().sort(desc(partition_name)).take(1)
    return archive[0].ARCHIVE

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TRANSFORMACIONES

In [31]:
def calculo_score(df_vars):

  df_vars = df_vars.fillna(0, subset=['cant_vehic','ind_interd','ind_interd','vehi_avaluo','avaluo_ind','sumc'])
  df_vars = df_vars.fillna('SIN CLASIFICACION', subset=['ise_equifax'])

  df_vars = (df_vars.withColumn('ise_abc1' ,when(col('ise_equifax') == 'ABC1',lit(1)).otherwise(lit(0)))
            .withColumn('ise_c2' ,when(col('ise_equifax') == 'C2',lit(1)).otherwise(lit(0)))
            .withColumn('ise_c3' ,when(col('ise_equifax') == 'C3',lit(1)).otherwise(lit(0)))
            .withColumn('ise_d' ,when(col('ise_equifax') == 'D',lit(1)).otherwise(lit(0)))
            .withColumn('ise_e' ,when(col('ise_equifax') == 'E',lit(1)).otherwise(lit(0)))
            .withColumn('profesion_ind' ,when(~col('codigo_prof').isin([0, 24]),lit(1)).otherwise(lit(0)))
            .withColumn('actividad_ind' ,when(~col('codigo_acti').isin([0, 24]),lit(1)).otherwise(lit(0)))
            .withColumn('sumc' ,when(col('sumc') <= 1,lit(0)).when(col('sumc') <= 2,lit(1)).otherwise(lit(2)))
            .withColumn('region_n_01' ,when(col('region') == 'DE TARAPACA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_02' ,when(col('region') == 'DE ANTOFAGASTA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_03' ,when(col('region') == 'DE ATACAMA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_04' ,when(col('region') == 'DE COQUIMBO',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_05' ,when(col('region') == 'DE VALPARAISO',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_06' ,when(col('region') == 'DEL LIBERTADOR BERNARDO OHIGGINS',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_07' ,when(col('region') == 'DEL MAULE',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_08' ,when((col('region') == 'DEL BIO BIO') | (col('region') == 'DE NUBLE'),lit(1)).otherwise(lit(0)))
            .withColumn('region_n_09' ,when(col('region') == 'DE LA ARAUCANIA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_10' ,when(col('region') == 'DE LOS LAGOS',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_11' ,when(col('region') == 'AYSEN DEL GENERAL CARLOS IBANEZ',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_12' ,when(col('region') == 'DE MAGALLANES Y ANTARTICA CHILENA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_14' ,when(col('region') == 'DE ARICA y PARINACOTA',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_15' ,when(col('region') == 'DE LOS RIOS',lit(1)).otherwise(lit(0)))
            .withColumn('region_n_RM' ,when(col('region') == 'METROPOLITANA DE SANTIAGO',lit(1)).otherwise(lit(0)))
            .withColumn('sexo_ind_f' ,when(col('sexo') == 'F',lit(1)).otherwise(lit(0)))
        )

  df_vars = (df_vars.withColumn('lineal_naf', -1.1755378016 + 
                                -0.3422733619 * col('sumc') + 
                                0.0189694125 * col('edad_calc') + 
                                0.4961972439 * col('ise_abc1') + 
                                0.6372084443 * col('ise_c2') + 
                                0.5321028263 * col('ise_c3') + 
                                0.2146062278 * col('ise_d') + 
                                0.0047105015 * col('ise_e') + 
                                0.7866098098 * col('clean2') + 
                                0.1941353042 * col('cant_vehic') + 
                                0.314980126 * col('sexo_ind_f') + 
                                -0.5345812623 * col('region_n_01') + 
                                -0.5518579373 * col('region_n_02') + 
                                -0.2392643935 * col('region_n_03') + 
                                0.2152253495 * col('region_n_04') + 
                                -0.1008099624 * col('region_n_05') + 
                                -0.2066496633 * col('region_n_06') + 
                                -0.1484280563 * col('region_n_07') + 
                                -0.0978616588 * col('region_n_08') + 
                                -0.1066844698 * col('region_n_09') + 
                                -0.2438581604 * col('region_n_10') + 
                                -0.9763161933 * col('region_n_11') + 
                                0.0963783118 * col('region_n_12') + 
                                -0.4401210306 * col('region_n_14') +
                                -0.1183049138 * col('region_n_15') +
                                -0.3276522064 * col('region_n_RM') +
                                0.2008862997 * col('avaluo_ind') +
                                0.241655416 * col('profesion_ind') + 
                                0.1026000436 * col('actividad_ind') +
                                0.88)
            )

  df_vars = (df_vars.withColumn('score_lp' , least(greatest(round(1000 / (1 + exp(-col('lineal_naf')))),lit(1)),lit(999)))
                    .withColumn('score_lp' , when((col('ind_defunc') == 1) | (col('ind_interd') == 1) | (col('edad_calc') < 18) , lit(0)).otherwise(col('score_lp')))
        )

  return df_vars

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LLAMAR TABLAS PARA UTILIZAR

In [32]:
#CONSOLIDADO

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consolidado")

query_consolidado = """
     SELECT rut_creado as rut
            ,1 as ind_consolidado
            ,coalesce(edad_calc, 0) as edad_calc
            ,coalesce(consultas_3m, 0) as sumc
            ,clean2 
            ,sexo
            ,fnac_26
            ,ind_interd
            ,ind_defunc
            ,ise_equifax
            ,region
            ,cast(codigo_acti as int) as codigo_acti
            ,cast(codigo_prof as int) as codigo_prof
            ,vehi_avaluo
            ,cant_vehic
            ,avaluo_tas_ind as avaluo_ind
            ,Null as Prueba 
     FROM consolidado
"""

df_consolidado = spark.sql(query_consolidado)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_consolidado.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------------+---------+----+------+----+----------+----------+----------+-----------------+--------------------+-----------+-----------+-----------+----------+----------+------+
|rut|ind_consolidado|edad_calc|sumc|clean2|sexo|   fnac_26|ind_interd|ind_defunc|      ise_equifax|              region|codigo_acti|codigo_prof|vehi_avaluo|cant_vehic|avaluo_ind|Prueba|
+---+---------------+---------+----+------+----+----------+----------+----------+-----------------+--------------------+-----------+-----------+-----------+----------+----------+------+
|  1|              1|       66|   0|     1|   M|1955-08-06|         0|         0|                D|METROPOLITANA DE ...|          2|          0|          0|         0|         0|  null|
|  2|              1|       32|   0|     1|   M|1989-09-09|         0|         0|               C3|METROPOLITANA DE ...|          0|          0|          0|         0|         0|  null|
|  3|              1|       50|   0|     0|   M|1971-07-05|         0|

## INFORMACION CONFIGURE

In [34]:
# ORIGIN DB AND TABLE NAMES
cl_efx_att = 'cl_efx_att'
cl_efx_prod = 'cl_efx_prod'
tb_relaciones_familiares = 'relaciones_familiares'
tb_consolidado = 'consolidado'
tb_consolidado_empresa = 'consolidado_empresa'
tb_personas_semanal = 'personas_semanal'
tb_empresas_semanal = 'empresas_semanal'
tb_hh_principal_hist = 'hh_principal_hist'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## CRUCE DE INFORMACION

In [35]:
df = calculo_score(df_consolidado)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
from pyspark.sql.types import NullType
import pyspark.sql.functions as F

# Check each column type. If it's nulltype, cast to string type,
# else keep the original column.

df2 = df.select([
    F.lit(None).cast('string').alias(i.name)
    if isinstance(i.dataType, NullType)
    else i.name
    for i in df.schema
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ESCRIBIR DATOS

In [39]:
df2.coalesce(1).write.mode('overwrite').csv('gs://dfa-dna-ws0011-la-prd-sandbox/contingencia_la_polar_diaria/contingencia_la_polar_diaria')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…